# Putting it Together

In the previous section we introduced blocks, the fundamental building blocks of a pipeline in Bifrost.  Now we will demonstrate how blocks are connected together and some of the considerations.

<a href="https://colab.research.google.com/github/ledatelescope/bifrost/blob/master/tutorial/03_putting_it_together.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"></a>

In [1]:
%%capture install_log
# Import bifrost, but attempt to auto-install if needed (and we're running on
# Colab). If something goes wrong, evaluate install_log.show() in a new block
# to retrieve the details.
try:
  import bifrost
except ModuleNotFoundError:
  try:
    import google.colab
    !sudo apt-get -qq install exuberant-ctags libopenblas-dev software-properties-common build-essential
    !pip install -q contextlib2 pint simplejson scipy git+https://github.com/ctypesgen/ctypesgen.git
    ![ -d ~/bifrost/.git ] || git clone https://github.com/ledatelescope/bifrost ~/bifrost
    !(cd ~/bifrost && ./configure && make -j all && sudo make install)
    import bifrost
  except ModuleNotFoundError:
    print("Sorry, could not import bifrost and we're not on colab.")

In Bifrost blocks are connected together by circular memory buffers called "rings".  Like a `bifrost.ndarray`, a ring exists in a memory space:  `system`, `cuda_host`, or `cuda`.  A ring also has a size that is based on a integer number of segments of the gulp size for the ring.

To create a ring in Bifrost:

In [2]:
ring = bifrost.ring.Ring(name="a_ring", space="system")
print('name:', ring.name, ', space:', ring.space)

name: b'a_ring' , space: system


This creates a new ring, called `"a_ring"`, in the system memory space.  Although the ring has been created it does not yet have any memory allocated to it.  To allocate memory you `resize` it:

In [3]:
ring.resize(4096)

This sets the gulp size for the ring to 4096 bytes and this call sets the total ring size to four, 4096 byte buffer.  You can change the buffer fraction by adding in a second argument which is the total ring size.  For example, to increase the buffer size to five segments:

In [4]:
ring.resize(4096, 5*4096)

Resizing a ring is a data-safe process and the contents of the ring are preserved.

Rings in Bifrost are more than just a section of memory, though.  It has a few other attributes that make it useful for representing a stream of data:

 * a timetag that denotes when the stream of data starts
 * a header that stores metadata about the sequence
 * they support single writer/multi-reader access for branching pipelines

Let's use an example to look at these first two.  In this we will write some data to the ring:

In [5]:
import json, numpy, time

ring = bifrost.ring.Ring(name="another_ring", space="system")

with ring.begin_writing() as output_ring:
    time_tag = int(time.time()*1e9)
    hdr = {'time_tag':      time_tag,
           'metadata':      'here',
           'more_metadata': 'there'}
    hdr_str = json.dumps(hdr)
    
    gulp_size = 4096
    ring.resize(gulp_size, 5*gulp_size)
    
    with output_ring.begin_sequence(time_tag=hdr['time_tag'],
                                    header=hdr_str) as output_seq:
        for i in range(20):
            with output_seq.reserve(gulp_size) as output_span:
                data = output_span.data_view(numpy.int8)
                data[...] = (numpy.random.rand(gulp_size)\
                             *127).astype(numpy.int8)
                print(i, '@', data[:5])

0 @ [[120  29  45 ...  68 113  43]]
1 @ [[ 41  69  98 ...  65  62 112]]
2 @ [[ 16  77 117 ...  20  56  38]]
3 @ [[ 74  63 107 ...  82 101 122]]
4 @ [[ 30  76 106 ...  93  97  78]]
5 @ [[ 33 119 119 ...  67  12  32]]
6 @ [[106 101  24 ...  89 104  62]]
7 @ [[26  6  1 ... 75 32 17]]
8 @ [[101  66 103 ... 116  34 112]]
9 @ [[ 14  31  70 ...  23 109 119]]
10 @ [[101 119  26 ...   3  74  43]]
11 @ [[ 76 109  16 ...  31  46  66]]
12 @ [[ 10  42  90 ...  99  38 126]]
13 @ [[ 42  86  40 ... 109  57  96]]
14 @ [[ 13  61   5 ...  13   2 113]]
15 @ [[ 86  82  79 ...  98 119  17]]
16 @ [[ 12  66 117 ...  16  75  20]]
17 @ [[ 68  50   3 ... 114  22  95]]
18 @ [[ 58  98  60 ...  73 108  86]]
19 @ [[  6  56  76 ... 115 107  44]]


Here we:

 1. Ready the ring for writing with `ring.begin_writing()`.
 2. Once the ring is ready for writing, we define the time tag for the first sample and a dictionary of metadata.  The time tag is expected to be an integer and the dictionary is dumped to a JSON object.
 3. Start a "sequence" on the ring using that time tag and JSON object. 
  * In Bifrost a sequence is a stream of data with a single observational setup.
 4. Loop over spans, also called gulps, in the output sequence and writes data to the ring.
  * Writing uses a `data_view` of the span/gulp that exposes it as a `bifrost.ndarray`.

Reading from a ring follows a similar sequence:

In [6]:
for input_seq in ring.read(guarantee=True):
    hdr = json.loads(input_seq.header.tobytes())
    print(input_seq.time_tag)
    print(hdr)
    
    gulp_size = 4096
    
    i = -1
    for input_span in input_seq.read(gulp_size):
        i += 1
        if input_span.size < gulp_size:
            continue
        data = input_span.data_view(numpy.int8)
        print(i, '@', data[:10])

1656354526479179008
{'time_tag': 1656354526479179008, 'metadata': 'here', 'more_metadata': 'there'}
12 @ [[ 10  42  90 ...  99  38 126]]
13 @ [[ 42  86  40 ... 109  57  96]]
14 @ [[ 13  61   5 ...  13   2 113]]
15 @ [[ 86  82  79 ...  98 119  17]]
16 @ [[ 12  66 117 ...  16  75  20]]
17 @ [[ 68  50   3 ... 114  22  95]]
18 @ [[ 58  98  60 ...  73 108  86]]
19 @ [[  6  56  76 ... 115 107  44]]


Here we:

 1. Open the ring for reading with `ring.read()` and get an iterator over sequences in that ring.
  * This ring was opened with `gaurantee=True` which tells Bifrost that spans that are being read from cannot be overwriten with new data until the reader releases the span.
 2. For the sequence we can access its `time_tag` and metadata header.
 3. Loop over spans/gulps within that sequence until the iterator is exhausted.
  * It is possible that a span returned by `input_seq.read()` is smaller than the gulp size, particuarlly at the end of a sequence.  It is a good idea to check the size of the span before trying to use it.
 4. For each span, do the processing that is required.

In the next section we will talk about how to build a complete pipeline from these pieces. 